In [1]:
%matplotlib inline

from pathlib import Path
from datetime import date

import numba
import rasterio
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs
from scipy import stats

import weather_ecmwf
import population_tools

from config import DATA_SRC, WEATHER_SRC, POP_DATA_SRC

In [21]:
for f in (WEATHER_SRC / 'ecmwf' / 'daily_precipitation').glob('*_daily_precipitation.nc'):
    year = int(f.name.split('_')[0])
    print(year)
    ppt = xr.open_dataset(f,
                          chunks={'latitude': 122, 
                                  'longitude': 240})
    ppt = ppt.resample(time='1D').sum() * 1000 # in meters
    ppt.to_netcdf(WEATHER_SRC / 'ecmwf' / 'daily_total_precipitation'/ f'{year}_daily_precipitation.nc')
    

1986
2012
1981
2015
1982
1988
2016
1985
2011
2010
1984
2017
1989
1983
2014
1980
2013
1987
2001
1995
2006
1998
1992
2005
1991
2002
2008
1996
1997
2009
2003
1990
2004
1993
1999
2007
1994
2000


In [2]:
ppt_daily = xr.open_mfdataset(str(WEATHER_SRC / 'ecmwf' / 'daily_total_precipitation'/ '*_daily_precipitation.nc'), 
                              chunks={'latitude': 100, 
                                  'longitude': 100})

In [3]:
ppt_5day = ppt_daily.tp.rolling(time=5).sum()
ppt_5day.name = 'precipitation'
ppt_5day = ppt_5day.to_dataset()

In [4]:
years, datasets = zip(*ppt_5day.groupby('time.year'))
paths = [WEATHER_SRC / 'ecmwf' / '5day_precipitation'/ f'{y}_ppt_5day.nc' for y in years]
xr.save_mfdataset(datasets, paths)